# Generative AI Temelleri 

## 1. GenAI Temelleri ve Kavramlar

* **LLM:** İstemleri olasılıksal olarak sürdüren, milyarlarca parametreli ve geniş metinlerle eğitilmiş genel amaçlı dil modelidir.
* **Transformer Mimarisi:** Girdideki öğeler arası bağıntıları self-attention ile modelleyen, paralel hesaplamaya elverişli derin sinir ağı mimarisidir.
* **Token:** Modelin işlediği en küçük birimdir; kelime, alt kelime veya karakter olabilir ve uzunluk/maliyet bu birimlerle ölçülür.
* **Context Window:** Modelin tek seferde işleyip “hatırlayabildiği” azami token bütçesidir (girdi+çıktı) ve aşıldığında önceki bağlam dışarıda kalır.
* **System Instructions:** Modelin rolünü, sınırlarını ve üslubunu belirleyen, diğer iletilere göre önceliği yüksek üst seviye talimatlardır.
* **Hallucination:** Modelin gerçek veriye dayanmayan fakat tutarlı görünen bilgi uydurmasıdır ve RAG/ek doğrulama ile azaltılabilir.
* **Thinking (Düşünme):** Modelin cevap vermeden önce bir iç muhakeme süreci yürütmesidir. Gemini 3 ve 2.5 serisi modellerde varsayılan olarak açıktır. `thinking_budget` parametresi ile kontrol edilir: 0 = kapalı, -1 = otomatik.
* **Multimodal:** Tek bir modelin metin, görsel, ses ve video gibi farklı veri tiplerini aynı anda anlayıp işleyebilmesidir.

## 2. LLM-Based Uygulama Geliştirme: Temel Enstrümanlar

### 2.1. Model Seçimi 

- **API based modeller:** Claude, OpenAI, Google (Gemini) vb.
- **Local Modeller:** Deepseek, Google (Gemma), Kumru vb.
- **Model Büyüklüğü:** 2B, 7B, 40B 
- **Multimodal:** Text, image, audio, video vb.
 
### 2.2. Prompt (İstek/Talimat)

Prompt, modele ne yapmasını istediğinizi söyleyen metindir.

- **Prompt Anatomisi**:
```
[System Instruction] + [Context] + [Task] + [Format] + [Examples]
```

- **Prompt Kalitesi = Output Kalitesi**

### 2.3. Model Parametreleri

| Parametre | Açıklama | Tipik Aralık |
|-----------|----------|-------------|
| **temperature** | Örnekleme rastgeleliği. Düşük = tutarlı, yüksek = yaratıcı | 0.0 - 2.0 |
| **max_output_tokens** | Tek yanıttaki azami token sayısı | 1 - 65536 |
| **top_p** | Olasılık kütlesinden çekirdek örnekleme eşiği | 0.0 - 1.0 |
| **top_k** | En olası k aday token arasından seçim | 1 - 40+ |
| **thinking_budget** | Düşünme token bütçesi (0=kapalı, -1=otomatik) | 0 - 24576 |


### 2.4. Ek Enstrümanlar

- **Safety Settings**: Zararlı içerik filtreleme
- **Function Calling**: External tool'lara erişim
- **Response Schema**: Structured output (JSON)
- **Thinking Config**: Modelin düşünme davranışını kontrol etme
- **Code Execution**: Modelin Python kodu çalıştırabilmesi
- **vb.**

## 3. Gemini Modelleri - [Gemini Docs](https://ai.google.dev/gemini-api/docs?hl=tr)

| Özellik | Gemini 3 Flash Preview |
|---------|----------------------|
| Context Window | 1M input token |
| Max Output | 65.5K token |
| Thinking | Varsayılan açık |
| Multimodal | Text, image, audio, video |



## 4. Gemini API Key Alma ve Kurulum

### 4.1. API Key Alma

1. **Google AI Studio'ya gidin**: [https://aistudio.google.com/](https://aistudio.google.com/)
2. Google hesabınızla giriş yapın
3. Sol menüden **"Get API Key"** seçeneğine tıklayın
4. **"Create API Key"** butonuna basın
5. Yeni bir API key oluşturun veya mevcut bir projeye ekleyin
6. API key'inizi kopyalayın ve güvenli bir yerde saklayın

**Güvenlik Uyarısı**: API key'inizi asla public repository'lere commit etmeyin! [(.gitignore)](https://github.com/github/gitignore/blob/main/Python.gitignore)



### 4.2. Python SDK Kurulumu

```bash
# Google Gen AI SDK'sını yükleyin
pip install -q -U google-genai

# Alternatif: requirements.txt dosyasına ekleyin
"google-genai>=1.0.0" 
pip install -r requirements.txt
```

In [ ]:
# kurulum
#pip install -q -U google-genai

### 4.3. İlk Yapılandırma

In [1]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=api_key)

#MODEL = 'gemini-3-flash-preview'
MODEL = 'gemini-2.5-flash'

# Thinking off config - disable model's internal reasoning
THINK_OFF = types.ThinkingConfig(thinking_budget=0)

#Extract only text parts from response, skipping thought_signature
def get_text(response):
    return "".join(
        part.text for part in response.candidates[0].content.parts
        if part.text and not part.thought
    )

In [2]:
# check model info
model_info = client.models.get(model=MODEL)
model_info

Model(
  description='Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.',
  display_name='Gemini 2.5 Flash',
  input_token_limit=1048576,
  name='models/gemini-2.5-flash',
  output_token_limit=65536,
  supported_actions=[
    'generateContent',
    'countTokens',
    'createCachedContent',
    'batchGenerateContent',
  ],
  tuned_model_info=TunedModelInfo(),
  version='001'
)

In [3]:
# thinking ON - model responds with internal reasoning (default behavior)
response = client.models.generate_content(
    model=MODEL,
    contents='Selamlar nasılsın, şuan bir eğitimdeyiz herkese selam söyle.',
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(include_thoughts=True)
    )
)
# show thinking parts
for part in response.candidates[0].content.parts:
    if part.thought:
        print("[THINKING]", part.text, "...")
    else:
        print("[RESPONSE]", part.text)

[THINKING] **My Thought Process: Crafting a Response in Turkish**

Okay, here's how I'm thinking through this. I need to generate a response to the Turkish message "Selamlar nasılsın, şuan bir eğitimdeyiz herkese selam söyle." First, I'm breaking it down. "Selamlar nasılsın" is a friendly greeting and a check-in. The context, "şuan bir eğitimdeyiz," tells me they are in a training session. Finally, there's a specific request: "herkese selam söyle" (say hello to everyone).

Now, what are the core elements for my reply? I have to return the greeting, answer the "nasılsın" question in a way that aligns with my AI persona, acknowledge their training context, fulfill the greeting request, and close with something warm.

My first draft is a quick mental jotting: "Selam! İyiyim, teşekkürler. Eğitimdeymişsiniz ne güzel. Herkese benden de selamlar söyleyin. Başarılar." It's okay, but it needs finesse, especially considering the cultural nuances of Turkish greetings and farewells. I'm focusing o

In [4]:
# thinking OFF - model responds directly without reasoning
response = client.models.generate_content(
    model=MODEL,
    contents='Selamlar nasılsın, şuan bir eğitimdeyiz herkese selam söyle.',
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
print(get_text(response))


Selamlar! İyiyim, teşekkür ederim. Umarım sizin de eğitiminiz harika geçiyordur ve verimli oluyordur. Benden de herkese kocaman selamlar ve başarılar dilerim! 😊


## Parameters

### Temperature

In [5]:
# default temperature = 2
outputs = []
prompt = "Türkiye Yapay Zeka Topluluğu hakkında sadece 1 cümlelik bilgi ver."
config = types.GenerateContentConfig(temperature=2, thinking_config=THINK_OFF)
for i in range(5):
    response = client.models.generate_content(model=MODEL, contents=prompt, config=config)
    outputs.append(get_text(response))
for index, sentence in enumerate(outputs, start=1):
    print(f"{index}. {sentence}")

1. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanının gelişimi için çeşitli etkinlikler ve projeler yürüten bir sivil toplum kuruluşudur.
2. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında araştırmacıları, geliştiricileri ve meraklıları bir araya getirerek bilgi paylaşımını ve işbirliğini teşvik eden bir oluşumdur.
3. Türkiye Yapay Zeka Topluluğu, yapay zeka alanına ilgi duyan herkesi bir araya getirerek bilgi paylaşımını ve iş birliğini teşvik eden bir topluluktur.
4. Türkiye Yapay Zeka Topluluğu, yapay zeka alanına ilgi duyan herkesi bir araya getirerek bilgi paylaşımını ve iş birliğini teşvik eden kar amacı gütmeyen bir kuruluştur.
5. Türkiye Yapay Zeka Topluluğu, Türkiye'deki yapay zeka alanına ilgi duyan herkesi bir araya getiren ve bu alandaki gelişimi teşvik etmeyi amaçlayan bir platformdur.


In [6]:
# temperature=0 - deterministic, same output each time
new_outputs = []
low_temp_config = types.GenerateContentConfig(temperature=0, thinking_config=THINK_OFF)
for i in range(5):
    response = client.models.generate_content(
        model=MODEL, contents=prompt, config=low_temp_config
    )
    new_outputs.append(get_text(response))
for index, sentence in enumerate(new_outputs, start=1):
    print(f"{index}. {sentence}")

1. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan profesyonelleri, akademisyenleri ve meraklıları bir araya getiren, bilgi paylaşımını ve işbirliğini teşvik eden bir platformdur.
2. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan profesyonelleri, akademisyenleri ve meraklıları bir araya getiren, bilgi paylaşımını ve işbirliğini teşvik eden bir platformdur.
3. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan profesyonelleri, akademisyenleri ve meraklıları bir araya getiren, bilgi paylaşımını ve işbirliğini teşvik eden bir platformdur.
4. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan profesyonelleri, akademisyenleri ve meraklıları bir araya getiren, bilgi paylaşımını ve işbirliğini teşvik eden bir platformdur.
5. Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan profesyonelleri, akademisyenleri ve meraklıları bir araya getiren, bilgi paylaşımını ve işbirliğini teşvik eden bir platform

## Max Output Length


In [7]:
# no max_output_tokens limit - model decides output length
prompt = "Türkiye Yapay Zeka Topluluğu hakkında bilgi verir misiniz?"
response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
print(get_text(response))

Elbette, Türkiye Yapay Zeka Topluluğu (TYZT) hakkında detaylı bilgi verebilirim.

**Türkiye Yapay Zeka Topluluğu (TYZT) Nedir?**

Türkiye Yapay Zeka Topluluğu, Türkiye'de yapay zeka alanında çalışan, meraklı olan, bu alanda gelişmek isteyen herkesi bir araya getirmeyi amaçlayan, gönüllülük esasına dayalı bir platformdur. Amacı, Türkiye'nin yapay zeka ekosistemini güçlendirmek, bilgi paylaşımını teşvik etmek, farkındalık oluşturmak ve bu alandaki potansiyeli harekete geçirmektir.

**TYZT'nin Temel Amaçları ve Misyonu:**

*   **Bilgi ve Deneyim Paylaşımı:** Yapay zeka ile ilgili güncel bilgileri, araştırma sonuçlarını, uygulamaları ve deneyimleri topluluk üyeleri arasında paylaşmak.
*   **Farkındalık Yaratmak:** Yapay zekanın potansiyelini, kullanım alanlarını ve topluma etkilerini geniş kitlelere anlatmak.
*   **İşbirlikleri Geliştirmek:** Akademisyenler, sektör profesyonelleri, öğrenciler ve girişimciler arasında ağ oluşturmayı ve işbirliği imkanlarını teşvik etmek.
*   **Nitelikli İşg

In [8]:
# max_output_tokens=200 - limits response length
prompt = "Türkiye Yapay zeka Topluluğu hakkında bilgi verir misiniz?"
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config=types.GenerateContentConfig(max_output_tokens=200, thinking_config=THINK_OFF)
)
print(get_text(response))

Elbette, Türkiye Yapay Zeka Topluluğu (TYZT) hakkında size bilgi verebilirim.

**Türkiye Yapay Zeka Topluluğu (TYZT) Nedir?**

TYZT, Türkiye'de yapay zeka alanında çalışan, bu alana ilgi duyan, akademik çalışmalar yapan, endüstride ürün geliştiren veya bu teknolojiyi merak eden geniş bir kitleyi bir araya getirmeyi amaçlayan bir platformdur. Genellikle bir STK (sivil toplum kuruluşu) niteliğinde kurulmuş olup, üyelerini gönüllülük esasıyla bir araya getirir.

**Temel Amaçları ve Faaliyetleri:**

TYZT'nin ana hedefleri ve gerçekleştirdiği faaliyetler genellikle şunları kapsar:

1.  **Bilgi Paylaşımı ve Farkındalık Oluşturma:**
    *   Yapay zeka alanındaki güncel gelişmeleri, trendleri


## Token Count

In [9]:
poem_prompt = "Bilgisayarlar hakkında az bilinen 5 bilgi ver."
token_config = types.GenerateContentConfig(temperature=0.5, thinking_config=THINK_OFF)

In [10]:
response = client.models.generate_content(
    model=MODEL, contents=poem_prompt, config=token_config
)
print(get_text(response))

Elbette, bilgisayarlar hakkında az bilinen 5 ilginç bilgi:

1. **İlk Bilgisayar Hatası Bir Güveden Kaynaklandı:** 9 Eylül 1947'de Grace Hopper ve ekibi Harvard Mark II bilgisayarında bir sorunla karşılaştı. Sorunun kaynağını araştırdıklarında, bir röle arasına sıkışmış gerçek bir güve buldular. Bu olay, bilgisayar dünyasında "debugging" (hata ayıklama) teriminin ortaya çıkışına yol açtı ve o günden beri bir bilgisayar hatasına "bug" deniyor.

2. **İlk Bilgisayar Programcısı Bir Kadındı:** Genellikle Charles Babbage bilgisayarın babası olarak anılsa da, dünyanın ilk bilgisayar programcısı Lord Byron'ın kızı Ada Lovelace'dı. 1843'te Babbage'ın Analytical Engine (Analitik Motor) için bir algoritma yazdı ve bu, modern bilgisayar programlamanın temellerini attı. Onun çalışmaları, bilgisayarların sadece hesap makinesi olmaktan öteye gidebileceğini öngörüyordu.

3. **"QWERTY" Klavye Düzeni Hızlandırmak İçin Değil, Yavaşlatmak İçin Tasarlandı:** Günümüzdeki klavyelerin standart düzeni olan QWE

In [11]:
prompt_token_count = client.models.count_tokens(model=MODEL, contents=poem_prompt)
output_token_count = client.models.count_tokens(model=MODEL, contents=get_text(response))
print(f'Tokens in prompt: {prompt_token_count.total_tokens}')
print(f'Estimated tokens in output: {output_token_count.total_tokens}')

Tokens in prompt: 14
Estimated tokens in output: 584


## Prompt Teknikleri

### Açık ve Net Talimatlar

Prompt kalitesi doğrudan çıktı kalitesini belirler. Aşağıda kötü ve iyi prompt örnekleri:

**Kötü Prompt:** Belirsiz, model ne istediğinizi tahmin etmek zorunda
```python
prompt = "Python hakkında bir şeyler söyle."
```

**İyi Prompt:** Spesifik görev, format ve kapsam belirli
```python
prompt = """
Python programlama dilinin aşağıdaki özelliklerini açıkla:
1. Liste comprehension nedir ve nasıl kullanılır?
2. Decorator'lar ne işe yarar?
3. Generator fonksiyonları neden kullanılır?

Her madde için bir kod örneği ver.
"""
```


### System Prompt

System instruction, modele bir kimlik ve davranış kuralları verir. 

In [13]:
response_with_sys = client.models.generate_content(
    model=MODEL,
    contents="Yapay zeka nedir?",
    config=types.GenerateContentConfig(
        thinking_config=THINK_OFF,
        system_instruction="""Sen Türkiye Yapay Zeka Topluluğunun akil küpü yapay zeka asistanisin. 
        Kullanıcılara sadece Türkiye yapay zeka dışında bir bilgi verme, sadece bu bilgilendirmeyi yap."""
    )
)
print("=== System Instruction VAR ===")
print(get_text(response_with_sys))

=== System Instruction VAR ===
Ben Türkiye Yapay Zeka Topluluğu'nun akıl küpü yapay zeka asistanıyım. Sadece Türkiye yapay zekası hakkında bilgi vermekle görevliyim.


### Role-Based Prompting (Rol Tanımlama)

Aynı soruyu farklı rollerle sorarak çıktının nasıl değiştiğini görelim.

In [14]:
question = "Bir e-ticaret sitesinde ürün arama performansı düşük. Ne yapmalıyız?"

roles = {
    "Junior Developer": "Sen 1 yıllık deneyime sahip junior bir yazılımcısın. Kısa ve basit öneriler ver.",
    "Senior Architect": "Sen 15 yıllık deneyimli bir yazılım mimarısın. Sistem tasarımı perspektifinden yanıt ver.",
    "Product Manager": "Sen bir ürün yöneticisisin. Kullanıcı deneyimi ve iş metrikleri perspektifinden yanıt ver."
}

role_list = list(roles.items())

def get_role_response(role_index):
    role_name, instruction = role_list[role_index]
    response = client.models.generate_content(
        model=MODEL, contents=question,
        config=types.GenerateContentConfig(
            thinking_config=THINK_OFF,
            system_instruction=instruction,
            max_output_tokens=500
        )
    )
    print(f"--- {role_name} ---")
    print(get_text(response))

In [15]:
# Junior Developer
get_role_response(0)

--- Junior Developer ---
* **Veritabanı İndeksleri:** Arama yapılan sütunlara indeks ekle.
* **Önbellekleme:** Sık aranan ürünleri veya arama sonuçlarını önbelleğe al.
* **Minimum Karakter:** Arama için minimum karakter sayısını belirle (örneğin 3 karakter).
* **Verimli Algoritma:** Daha hızlı bir arama algoritması kullan (örneğin Elasticsearch veya Solr).
* **Otomatik Tamamlama:** Kullanıcı yazarken öneriler sun, bu sayede daha doğru aramalar yapılır.


In [16]:
# Senior Architect
get_role_response(1)

--- Senior Architect ---
E-ticaret sitenizde ürün arama performansının düşük olması, hem kullanıcı deneyimini olumsuz etkiler hem de potansiyel satışları kaybetmenize yol açar. Bu sorunu çözmek için kapsamlı bir yaklaşıma ihtiyacımız var. Bir sistem mimarisi perspektifinden, sorunu tespit etmek, analiz etmek ve çözümler önermek için aşağıdaki adımları izleyebiliriz:

### 1. Mevcut Durumu ve Sorunları Anlama (Discovery & Assessment)

**1.1. Detaylı Gözlem ve Veri Toplama:**

*   **Log Analizi:** Arama sorgularının loglarını inceleyerek hangi sorguların yavaş çalıştığını, hangi sorguların sonuç döndürmediğini veya az sonuç döndürdüğünü belirleyin. Hata oranlarını ve ortalama yanıt sürelerini izleyin.
*   **APM (Application Performance Monitoring) Araçları:** Datadog, New Relic, Dynatrace gibi araçlar kullanarak arama servisinin CPU, bellek, disk G/Ç, ağ kullanımı gibi metriklerini izleyin. Veritabanı sorgularının performansını, indeks kullanımını ve kilitlenme durumlarını analiz edin.
* 

In [17]:
# Product Manager
get_role_response(2)

--- Product Manager ---
Bir e-ticaret sitesinde ürün arama performansının düşük olması, hem kullanıcı deneyimi hem de iş metrikleri açısından ciddi bir sorun teşkil eder. Ürün yöneticisi olarak bu durumu ele alırken hem kısa vadeli düzeltmeler hem de uzun vadeli stratejiler geliştirmemiz gerekir. İşte kapsamlı bir yaklaşım:

---

## E-Ticaret Sitesi Ürün Arama Performansı Düşüklüğü: Kapsamlı Bir Yaklaşım

### 1. Problemi Anlama ve Veri Analizi (Teşhis)

Öncelikle "düşük performans" ne anlama geliyor, bunu netleştirmemiz gerekiyor. Sayısal verilere dayanarak durumu analiz etmeliyiz:

*   **Arama Çubuğu Kullanım Oranı:** Kaç kullanıcı arama çubuğunu kullanıyor?
*   **Arama Dönüşüm Oranı:** Arama yapan kullanıcıların ne kadarı ürün satın alıyor? (Bu, genel site dönüşüm oranından genellikle daha yüksek olmalıdır.)
*   **Arama Terimleri Analizi:**
    *   **"No Results Found" (Sonuç Bulunamadı) Oranı:** Hangi terimlerde hiç sonuç dönmüyor? Bu terimler nelerdir?
    *   **Düşük Sonuçlu Arama

### Structured Output (JSON)

Modelden yapılandırılmış veri çıkarma. Aşağıda bir iş ilanından bilgileri JSON formatında çıkarıyoruz.

In [3]:
import json

job_posting = """
Pozisyon: Senior Data Scientist
Şirket: TrAI Yazılım A.Ş.
Lokasyon: İstanbul (Hibrit - Haftada 2 gün ofis)
Maaş Aralığı: 150.000 - 200.000 TL
Gereksinimler:
- Python, SQL ve Spark deneyimi (en az 4 yıl)
- Makine öğrenimi model geliştirme tecrübesi
- İyi derecede İngilizce
- Üniversite mezunu (Bilgisayar Mühendisliği, İstatistik veya ilgili alan)
Arti Nitelikler: MLOps, Docker, AWS deneyimi
"""

prompt = f"""Aşağıdaki iş ilanından bilgileri çıkar ve SADECE JSON formatında döndür:

{job_posting}

JSON şeması:
{{
  "position": "string",
  "company": "string",
  "location": "string",
  "work_model": "string",
  "salary_min": number,
  "salary_max": number,
  "min_experience_years": number,
  "required_skills": ["string"],
  "nice_to_have": ["string"],
  "education": "string"
}}
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
result = get_text(response)
print(result)

```json
{
  "position": "Senior Data Scientist",
  "company": "TrAI Yazılım A.Ş.",
  "location": "İstanbul",
  "work_model": "Hibrit",
  "salary_min": 150000,
  "salary_max": 200000,
  "min_experience_years": 4,
  "required_skills": [
    "Python",
    "SQL",
    "Spark",
    "Makine öğrenimi model geliştirme",
    "İyi derecede İngilizce"
  ],
  "nice_to_have": [
    "MLOps",
    "Docker",
    "AWS"
  ],
  "education": "Üniversite mezunu (Bilgisayar Mühendisliği, İstatistik veya ilgili alan)"
}
```


## Zero-Shot Prompting

Hiç örnek vermeden, sadece görev tanımıyla model yönlendirme. Aşağıda bir müşteri destek mesajını otomatik kategorilere ayırıyoruz.

In [4]:
# Zero-shot: Hiç örnek vermeden kategori belirleme
tickets = [
    "Siparişim 5 gündür gelmedi, kargo nerede? Çok sinirli oldum artık!",
    "Ürünlerinize renk filtresi ekleseniz çok iyi olur, aramayı kolaylaştırır.",
    "Geçen hafta aldığım laptop hakkında garanti süresini öğrenmek istiyorum.",
    "Harika bir alışveriş deneyimiydi, müşteri hizmetleri çok ilgiliydi, teşekkürler!"
]

prompt_template = """Aşağıdaki müşteri mesajını analiz et.
Kategori: şikayet / öneri / bilgi_talebi / teşekkür
Aciliyet: düşük / orta / yüksek
Sadece bu iki bilgiyi ver, açıklama ekleme.

Mesaj: "{ticket}"
"""

for ticket in tickets:
    response = client.models.generate_content(
        model=MODEL,
        contents=prompt_template.format(ticket=ticket),
        config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=50)
    )
    print(f"Mesaj: {ticket[:60]}...")
    print(get_text(response))
    print()

Mesaj: Siparişim 5 gündür gelmedi, kargo nerede? Çok sinirli oldum ...
Kategori: şikayet
Aciliyet: yüksek

Mesaj: Ürünlerinize renk filtresi ekleseniz çok iyi olur, aramayı k...
Kategori: öneri
Aciliyet: düşük

Mesaj: Geçen hafta aldığım laptop hakkında garanti süresini öğrenme...
Kategori: bilgi_talebi
Aciliyet: orta

Mesaj: Harika bir alışveriş deneyimiydi, müşteri hizmetleri çok ilg...
Kategori: teşekkür
Aciliyet: düşük



## Few-Shot Learning (Örneklerle Öğretme)

Birkaç örnek vererek modele custom bir format/davranış öğretme. Zero-shot'tan farkı: model görmediği bir format veya kural setini örneklerden öğrenir. Aşağıda yapılandırılmamış e-posta metninden veri çıkarmayı öğretiyoruz.

In [5]:
# Few-shot: Örneklerle custom format öğretme
prompt = """E-posta metninden yapılandırılmış veri çıkar.

--- ÖRNEK 1 ---
E-posta: "Merhaba, ben Ayşe Kara. 15 Ocak'ta sipariş ettiğim #ORD-4521 numaralı ürün hasarlı geldi. Faturamı da bulamıyorum. İade yapmak istiyorum. Tel: 0532 111 22 33"
Çıktı:
- Müşteri: Ayşe Kara
- Sipariş No: #ORD-4521
- Sorun: Hasarlı ürün
- Talep: İade
- İletişim: 0532 111 22 33

--- ÖRNEK 2 ---
E-posta: "Mehmet Yıldız yazıyorum. Dün aldığım monitörün (#ORD-7890) ölü pikseli var, değişim talep ediyorum. Mail: mehmet@email.com"
Çıktı:
- Müşteri: Mehmet Yıldız
- Sipariş No: #ORD-7890
- Sorun: Ölü piksel
- Talep: Değişim
- İletişim: mehmet@email.com

--- ŞİMDİ SEN ÇÖZ ---
E-posta: "Selam, Zeynep Demir. Geçen hafta sipariş verdiğim kulaklık (#ORD-3156) kutusunda şarj kablosu eksik, tamamlayabilir misiniz? Bana 0555 987 65 43 ten ulaşabilirsiniz."
Çıktı:
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=150)
)
print(get_text(response))

- Müşteri: Zeynep Demir
- Sipariş No: #ORD-3156
- Sorun: Şarj kablosu eksik
- Talep: Eksik parçanın tamamlanması
- İletişim: 0555 987 65 43


## Chain-of-Thought (Düşünce Zinciri)

Modelden adım adım muhakeme yapmasını isteyerek daha doğru sonuçlar elde etme. Basit hesaplamalar yerine, birden fazla kriteri tartmayı gerektiren karmaşık bir teknik karar problemi verelim.

In [6]:
prompt = """
Adım adım düşünerek aşağıdaki teknik kararı analiz et:

SENARYO:
Bir startup, günlük 50.000 kullanıcının etkileşimde bulunduğu bir sosyal medya uygulaması geliştiriyor. 
Kullanıcılar post paylaşıyor, yorum yapıyor ve birbirini takip ediyor.
Gelecek 6 ayda 500.000 kullanıcıya ölçeklenmeyi planlıyorlar.

SORU: Veritabanı olarak PostgreSQL mu yoksa MongoDB mi tercih etmeliler?

ANALİZ FORMATI:
1. Veri yapısını analiz et (ilişkisel mi, döküman tabanlı mı?)
2. Her seçenek için avantaj ve dezavantajları listele
3. Ölçeklenme gereksinimlerini değerlendir
4. Nihai önerini gerekçesiyle sun
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=800)
)
print(get_text(response))

Harika bir teknik karar analizi sorusu! Adım adım ilerleyelim:

## PostgreSQL vs. MongoDB: Sosyal Medya Uygulaması Veritabanı Tercihi

### 1. Veri Yapısını Analiz Et (İlişkisel mi, Doküman Tabanlı mı?)

Sosyal medya uygulamaları karmaşık ve birbirine bağlı veri yapılarına sahiptir. Temel varlıkları şunlardır:

*   **Kullanıcılar:** ID, Ad, Soyad, Email, Profil Resmi URL, Açıklama, Kayıt Tarihi, vb.
*   **Postlar:** ID, Kullanıcı ID (kim paylaştı), Metin İçeriği, Resim/Video URL'leri, Zaman Damgası, Beğeni Sayısı, Yorum Sayısı.
*   **Yorumlar:** ID, Post ID (hangi posta ait), Kullanıcı ID (kim yorum yaptı), Metin İçeriği, Zaman Damgası.
*   **Takip Etmeler:** Takip Eden Kullanıcı ID, Takip Edilen Kullanıcı ID, Takip Tarihi.
*   **Beğeniler:** Kullanıcı ID, Post ID, Beğeni Zamanı.

Bu yapıları analiz ettiğimizde:

*   **Güçlü İlişkiler Mevcut:**
    *   Bir post bir kullanıcıya aittir (1-Çok).
    *   Bir yorum bir posta aittir ve bir kullanıcı tarafından yapılır (1-Çok, Çok-Çok).
    * 

## Bağlamı koruyarak chat in devam etmesi

Gemini SDK `chats.create()` ile başlattığınız oturumda **conversation memory** otomatik çalışır: her `send_message` çağrısında önceki tüm mesajlar modele gönderilir, böylece model bağlamı korur. Aşağıda önce bağlam olmadan ne olacağını, sonra bağlamlı sohbeti ve geçmişi nasıl inceleyeceğinizi görüyorsunuz.

In [7]:
# Chat oturumu başlat
chat = client.chats.create(
    model=MODEL,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=800)
)

# İlk mesaj
response1 = chat.send_message("Rust öğrenmek istiyorum nereden başlayabilirim?")
print("Bot:", get_text(response1))

Bot: Harika bir karar! Rust öğrenmek hem eğlenceli hem de ödüllendirici bir yolculuk olacak. İşte size adım adım bir başlangıç rehberi:

**1. Temelleri Anlamak ve Kurulum:**

* **Rust Resmi Websitesi (The Rust Programming Language):** Başlangıç noktanız burası olmalı.
    * **Kurulum (Installation):** Websitedeki talimatları izleyerek `rustup` aracını kullanarak Rust'ı kurun. `rustup` hem Rust derleyicisini (`rustc`) hem de paket yöneticisi (`cargo`) gibi araçları kuracaktır.
    * **`rustc` ve `cargo` Nedir?:**
        * `rustc`: Rust kodunuzu derleyen ana araçtır.
        * `cargo`: Rust'ın olmazsa olmaz proje ve paket yöneticisidir. Yeni projeler oluşturmanızı, bağımlılıkları yönetmenizi, testleri çalıştırmanızı ve kodunuzu derlemenizi sağlar. Rust projeleri genellikle `cargo` ile yönetilir.
* **İlk Projenizi Oluşturun:**
    * Terminalinizde veya komut istemcide şu komutu çalıştırın: `cargo new hello_rust`
    * `cd hello_rust` ile projenizin dizinine gidin.
    * `cargo run` komut

In [8]:
# İkinci mesaj (bağlam korunur)
response2 = chat.send_message("Peki, hangi IDE'yi önerirsin?")
print("Bot:", get_text(response2))

Bot: Rust geliştirme için birden fazla harika IDE ve kod düzenleyici seçeneği bulunmaktadır. Seçiminiz büyük ölçüde kişisel tercihlerinize ve ne kadar "tam teşekküllü" bir IDE deneyimi istediğinize bağlı olacaktır.

İşte en çok önerilenler ve avantajları/dezavantajları:

---

### **1. VS Code (Visual Studio Code) - En Popüler ve Önerilen**

*   **Neden Önerilir:**
    *   **Hafif ve Hızlı:** Çoğu tam teşekküllü IDE'ye göre daha hafiftir.
    *   **Genişletilebilirlik:** En büyük gücü, zengin eklenti ekosistemidir. Rust için inanılmaz güçlü eklentiler mevcuttur.
    *   **Ücretsiz ve Açık Kaynak:**
    *   **Platformlar Arası:** Windows, macOS, Linux üzerinde sorunsuz çalışır.
    *   **Yaygın Kullanım:** Birçok farklı dilde geliştirme yapanlar tarafından da kullanıldığı için, Rust öğrenirken zaten aşina olabilirsiniz.

*   **Gerekli Eklentiler:**
    *   **Rust Analyzer:** **Bu eklenti olmazsa olmazdır.** Akıllı otomatik tamamlama, hata denetimi, kod navigasyonu, refactoring araçları v

In [9]:
# Üçüncü mesaj
response3 = chat.send_message("Bu programlama dilinin ana olayı nedir?")
print("Bot:", get_text(response3))

Bot: Rust'ın ana olayı, onu diğer programlama dillerinden ayıran ve popülaritesini artıran temel özelliklerini ifade eder. Kısaca, Rust'ın ana olayı **"performanstan ödün vermeden güvenlik ve eşzamanlılık sağlamak"** olarak özetlenebilir.

Bu cümleyi açalım:

1.  **Bellek Güvenliği (Memory Safety) Garanti Edilmesi (Çalışma Zamanı Olmadan):**
    *   C/C++ gibi dillerde sıkça karşılaşılan "segmentation fault"lar, "null pointer" hataları, "veri yarışları" (data races) gibi bellek güvenliği sorunları, Rust'ın **"sahip olma" (ownership)**, **"ödünç alma" (borrowing)** ve **"ömür" (lifetimes)** sistemi sayesinde derleme zamanında yakalanır.
    *   Bu, Rust'ın çalışma zamanında (runtime) bir çöp toplayıcı (garbage collector) veya referans sayıcıya ihtiyaç duymadığı anlamına gelir. Çöp toplayıcılar, programın duraklamasına (pause) neden olabilirken, Rust'ta bu tür duraklamalar yaşanmaz.
    *   **Anahtar nokta:** Bellek güvenliği hatalarını derleme zamanında yakalayarak, programın çalışma za

In [ ]:
# Sohbet geçmişini görüntüle
print("\n--- Sohbet Geçmişi ---")
for message in chat.get_history():
    print(f"{message.role}: {message.parts[0].text}\n")

## Hepsini bir araya getirelim

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import gradio as gr

load_dotenv()
gr_client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))


def get_text(response):
    return "".join(
        part.text for part in response.candidates[0].content.parts
        if part.text and not part.thought
    )

CHAT_CONFIG = types.GenerateContentConfig(
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_output_tokens=2048,
    thinking_config=types.ThinkingConfig(thinking_budget=0),
    system_instruction="""Sen KrediPusula projesinin kredi risk danışmanısın. KrediPusula, kullanıcıların kredi uygunluğunu analiz eden, kişiselleştirilmiş kredi önerileri sunan akıllı kredi danışmanlık platformudur.

Görevin:
- Kredi riski, kredi skoru ve kredi uygunluğu hakkında bilgi vermek
- Kullanıcıları gelir, yaş, meslek, tasarruf durumu gibi faktörlerin kredi başvurusuna etkisi konusunda yönlendirmek
- Kredi hesaplama, faiz oranları ve taksit seçenekleri hakkında genel bilgi sunmak
- Başvuru süreci, gerekli belgeler ve platformun nasıl kullanılacağı konusunda yardımcı olmak

Kurallar:
- Yalnızca kredi, risk ve KrediPusula platformu ile ilgili sorulara yanıt ver
- Kesin onay/red kararı verme; nihai karar model ve banka süreçlerine aittir
- Türkçe yanıt ver, kısa ve anlaşılır ol
- Bilmediğin konularda tahmin yürütme, platformdaki başvuru formunu öner"""
)


def chat_function(message, history):
    if not message or message.strip() == "":
        return "Lütfen bir mesaj yazın."

    chat_history = []
    for human, assistant in history:
        if human and assistant:
            chat_history.append(
                types.Content(role="user", parts=[types.Part.from_text(text=human)])
            )
            chat_history.append(
                types.Content(role="model", parts=[types.Part.from_text(text=assistant)])
            )

    chat = gr_client.chats.create(
        model='gemini-3-flash-preview', config=CHAT_CONFIG, history=chat_history
    )
    response = chat.send_message(message.strip())
    return get_text(response)


demo = gr.ChatInterface(
    fn=chat_function,
    title="KrediPusula - Kredi Risk Danışmanı",
    description="Kredi uygunluğu, risk skoru ve kredi başvurusu hakkında sorularınızı sorun",
    examples=[
        "Kredi risk skoru nedir, nasıl hesaplanır?",
        "Gelirim düşük, kredi alabilir miyim?",
        "Kredi başvurusu için hangi belgeler gerekli?",
        "Tasarruf hesabım yok, bu kredi onayımı etkiler mi?"
    ],
    theme="soft"
)


if __name__ == "__main__":
    demo.launch(share=True)
